<a href="https://colab.research.google.com/github/franconoronha/treinamento-h2ia/blob/main/Semana_3_BuscasComInformacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O Problema
Sliding Puzzle - Bloco Deslizante

In [ ]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

# Resolver o quebra-cabeças usando Busca Informada

In [120]:
import numpy as np
from random import seed, randint
from heapq import heapify, heappush, heappop

# Comentar a seed para ter um puzzle aleatório
seed(77)
# Seeds solucionáveis
# 101, 24, 133, 2, 77

class Tree:
  def __init__(self, data, parent, move, g):
    self.data = data
    self.parent = parent
    self.move = move
    self.h = get_heuristica(data)
    self.g = g

class Tree_2: # Estrutura pra testar a heuristica 2
  def __init__(self, data, parent, move, g):
    self.data = data
    self.parent = parent
    self.move = move
    self.h = get_heuristica_2(data)
    self.g = g

def gerar_puzzle(shape):
  puzzle = np.zeros((shape,shape), dtype=np.int64)
  numeros = [*range(1, (shape ** 2))]
  for x in range(0, len(numeros)):
    num = numeros.pop(randint(0, len(numeros) - 1))
    puzzle[int(x / 3)][x % 3] = num

  solucao = np.arange(1,shape**2 + 1).reshape((shape, shape))
  solucao[shape-1][shape-1] = 0
  return puzzle, solucao

def get_heuristica(puzzle):
  # Distância de Manhattan
  h = 0
  for coord, val in np.ndenumerate(puzzle):
    if val == 0:
      continue
    val -=1   # Os "nomes" das peças começam em 0 e as posições no array em 0
    d_h = abs(coord[1] - (val % 3))
    d_v = abs(coord[0] - int(val / 3))
    h += d_v + d_h
  return h

def get_heuristica_2(puzzle):
  h = 0
  for coord, val in np.ndenumerate(puzzle):
    if val == 0:
      continue
    val -=1 
    d_h = abs(coord[1] - (val % 3))
    d_v = abs(coord[0] - int(val / 3))
    if(d_h != 0 or d_v != 0) and val !=0: # se a peça não está no lugar correta
      d = d_h + d_v                       # pega a distância
      h += (d * 4)/val                    # atribui um peso (relativo a distância e a peça em questão)
      # O peso da peça é inversamente proporcional a posição dela 
  return h

def move(array_o, move):
  array = np.copy(array_o) # se manter o original, ele efetua as mudanças fora da função
  zero = np.argwhere(array == 0)[0]
  new_row = zero[0] + move[0]
  new_col = zero[1] + move[1]
  if (new_row >= 0 and new_row < array.shape[0]) and (new_col >= 0 and new_col < array.shape[1]):
    array[zero[0]][zero[1]], array[new_row][new_col] = array[new_row][new_col], array[zero[0]][zero[1]]
    return (array, True)
  else:
    return (None, False)

def gerar_estados(estado):
  novos_estados = []
  moves = [(0,1),(0,-1),(-1,0),(1,0)]
  for m in moves:
    novo_puzzle = move(estado.data, m)
    if novo_puzzle[1]: 
      novos_estados.append({
          "estado": novo_puzzle[0],
          "move": m
      })
  return novos_estados

puzzle, solucao = gerar_puzzle(3) # Gera puzzle e solução nxn
print("Puzzle gerado: \n{}".format(puzzle))

Puzzle gerado: 
[[5 3 2]
 [4 6 1]
 [8 7 0]]


## A*

In [123]:
def aplica_resultado(puzzle_o, passos):
  puzzle = np.copy(puzzle_o)
  for p in passos:
    puzzle = move(puzzle, p)[0]
  return puzzle

def a_estrela(puzzle):
  global solucao
  borda = []
  heapify(borda)
  gerados = set()
  passos = []
  arvore = Tree(puzzle, None, None, 0) # (data, parent, move, g)
  heappush(borda, (arvore.g + arvore.h, id(arvore), arvore)) # id(unico) desempata em caso de f(n) iguais
  gerados.add(puzzle.tobytes())    

  while len(borda) > 0:
      atual = heappop(borda)[2]
      if np.array_equal(atual.data, solucao):
          break
      for estado in gerar_estados(atual):
        if estado["estado"].tobytes() not in gerados:
          novo_nodo = Tree(estado["estado"], atual, estado["move"], atual.g + 1)
          val = novo_nodo.g + novo_nodo.h
          heappush(borda, (val, id(novo_nodo), novo_nodo))
          gerados.add(novo_nodo.data.tobytes())

  print("Estados gerados: {}".format(len(gerados)))
  while atual.parent != None:
    passos.append(atual.move)
    atual = atual.parent
  return passos[::-1]

resultado = a_estrela(puzzle)
print("Nº de passos: {}".format(len(resultado)))
print("Aplicando passos no puzzle:")
print(aplica_resultado(puzzle, resultado))

Estados gerados: 1085
Nº de passos: 22
Aplicando passos no puzzle:
[[1 2 3]
 [4 5 6]
 [7 8 0]]


### Testando outra heuristica

In [122]:
def a_estrela_2(puzzle):
  global solucao
  borda = []
  heapify(borda)
  gerados = set()
  passos = []
  arvore = Tree_2(puzzle, None, None, 0) # (data, parent, move, g)
  heappush(borda, (arvore.g + arvore.h, id(arvore), arvore)) # id(unico) desempata em caso de f(n) iguais
  gerados.add(puzzle.tobytes())    

  while len(borda) > 0:
      atual = heappop(borda)[2]
      if np.array_equal(atual.data, solucao):
          break
      for estado in gerar_estados(atual):
        if estado["estado"].tobytes() not in gerados:
          novo_nodo = Tree_2(estado["estado"], atual, estado["move"], atual.g + 1)
          val = novo_nodo.g + novo_nodo.h
          heappush(borda, (val, id(novo_nodo), novo_nodo))
          gerados.add(novo_nodo.data.tobytes())

  print("Estados gerados: {}".format(len(gerados)))
  while atual.parent != None:
    passos.append(atual.move)
    atual = atual.parent
  return passos[::-1]

resultado = a_estrela_2(puzzle)
print("Nº de passos: {}".format(len(resultado)))
print("Aplicando passos no puzzle:")
print(aplica_resultado(puzzle, resultado))

Estados gerados: 2199
Nº de passos: 24
Aplicando passos no puzzle:
[[1 2 3]
 [4 5 6]
 [7 8 0]]
